In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import random
import seaborn as sns
import scipy
sns.set()

In [284]:
import sys

sys.path

['/Users/seanmcalevey/Downloads',
 '/Users/seanmcalevey/anaconda3/lib/python37.zip',
 '/Users/seanmcalevey/anaconda3/lib/python3.7',
 '/Users/seanmcalevey/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/seanmcalevey/anaconda3/lib/python3.7/site-packages',
 '/Users/seanmcalevey/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/seanmcalevey/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/seanmcalevey/.ipython']

In [61]:
dict1 = {'a': 5, 'b': 1, 'c': 'ten'}

dict1.values()

dict_values([5, 1, 'ten'])

In [59]:
class Team:
    
    # League-wide stats/avgs
    league_average_ypc = 11.1
    league_average_ypc_std = 0.67
    league_average_ypr = 4.3
    league_average_ypr_std = 0.42
    league_average_comp_pct = 0.65
    league_average_comp_pct_std = 0.035
    league_average_int_pct = 0.029
    league_average_int_per_inc_pct = 0.083
    league_average_lost_fumble_pct = 0.01
    league_average_int_yds_lost = 25
    league_average_net_punt_yds = 40
    league_average_punt_std = 4.0
    league_average_sack_pct = 0.06
    league_average_sack_yds_lost = 6.5
    league_average_sack_std = 0.0
    league_average_yard_factors = [0.95, 0.66, 0.5, 0.25]
    league_average_down_factors = [0.95, 0.5, 0.2]
    game_duration = 3600
    std_play_duration = 7
    std_run_clock_duration = 33
    std_start_yardline = 25
    std_pass_yds_std = 11.5
    std_run_yds_std = 6.0
    base_lr = 0.001
    
    # Simple Class-lvl Funcs
    min_max_func = lambda self, x: max(max(x, 0.5), 4.0)
    flip_field_func = lambda self, x: 100 - x if x > 0 else 75
    
    
    def __init__(self, name, learning=False, off_ypc=None, def_ypc=None, off_ypr=None, def_ypr=None,
                 comp_pct=None, pnt_ypp=None, yd_factors=None, down_factors=None, run_pass_bias=None):
        
        # Direct
        self.name = name
        self.lr = self.base_lr if learning else None
        
        # YPP and OppYPP
        self.off_ypc = off_ypc if off_ypc else self.league_average_ypc
        self.def_ypc = def_ypc if def_ypc else self.league_average_ypc
        self.off_ypr = off_ypr if off_ypr else self.league_average_ypr
        self.def_ypr = def_ypr if def_ypr else self.league_average_ypr
        
        # Comp./Incomp. %
        self.real_comp_pct = comp_pct if comp_pct else self.league_average_comp_pct
        self.raw_inc_pct = 1 - comp_pct if comp_pct else 1 - self.league_average_comp_pct
        self.ovr_int_pct = self.league_average_int_pct * (self.league_average_comp_pct / self.real_comp_pct)
        self.real_inc_pct = self.raw_inc_pct - self.ovr_int_pct
        
        # Special Teams
        self.pnt_ypp = pnt_ypp if pnt_ypp else self.league_average_net_punt_yds
        
        # Yard Factors --> Playcall Matrix
        self.yd_factors = yd_factors if yd_factors else self.league_average_yard_factors
        self.down_factors = down_factors if down_factors else self.league_average_down_factors
        self.playcall_matrix = self.generate_playcall_matrix(self.yd_factors, self.down_factors, run_pass_bias)
        
        # Calculated Z Scores, INT Pct.
        self.off_ypc_z = (self.off_ypc - self.league_average_ypc) / self.league_average_ypc_std
        self.def_ypc_z = (self.def_ypc - self.league_average_ypc) / self.league_average_ypc_std
        self.off_ypr_z = (self.off_ypr - self.league_average_ypr) / self.league_average_ypr_std
        self.def_ypr_z = (self.def_ypr - self.league_average_ypr) / self.league_average_ypr_std
        
        # Set Initial Stats (Unchanging)
        self.init_off_ypc_z = self.off_ypc_z
        self.init_def_ypc_z = self.def_ypc_z
        self.init_off_ypr_z = self.off_ypr_z
        self.init_def_ypr_z = self.def_ypr_z
        
        # Reset all counting stats to 0
        self.set_stats_to_defaults()
        
    def play_opponent(self, other_team):
        '''Primary method to initiate a game with an opponent. This method will set all counting stats to 
        their defaults, and then initiate the first method in a series of recursive method calls terminating 
        at the end of the game (i.e., 3600 game-seconds).'''
        
        self.set_stats_to_defaults(other_team)
        return self.generate_off_play(other_team)
        
    def generate_off_play(self, other_team):
        
        # Check for Game End
        if (self.time_remaining <= 0) or (other_team.time_remaining <= 0):
            self.game_on, other_team.game_on = False, False
            score_diff = self.score - self.opp_score
            self.diff_tot = [off_play[1][0] - off_play[1][1] for off_play in self.off_plays]
            other_team.diff_tot = [off_play[1][0] - off_play[1][1] for off_play in self.opp_off_plays]
            
            # Update wins
            if score_diff > 0:
                winner = self
            elif score_diff == 0:
                winner = 'd'
            else:
                winner = other_team
            
            # Set single game stats
            self.game_ypp = self.yds_pass_plays / self.pass_plays
            self.game_ypr = self.yds_run_plays / self.run_plays
            self.game_def_ypc = other_team.yds_pass_plays / other_team.pass_plays
            self.game_def_ypr = other_team.yds_run_plays / other_team.run_plays
            
            other_team.game_ypp = other_team.yds_pass_plays / other_team.pass_plays
            other_team.game_ypr = other_team.yds_run_plays / other_team.run_plays
            other_team.game_def_ypc = self.yds_pass_plays / self.pass_plays
            other_team.game_def_ypr = self.yds_run_plays / self.run_plays
            
            # Adjust base team stats
            if self.lr:
                self.apply_lr_step(other_team, self.lr)
            
            return (winner, self.name, self, other_team)
        
        # Update Drive Stats
        if (self.plays_since_poss == 0) and (self.possession == True):
            self.drives += 1
            other_team.opp_drives += 1
        
        # Generate Playcall
        playcall = self.choose_playcall()
        
        # Generate Random Float in Range [0,1)
        rand_float = np.random.random()
        
        # Generate Simulated Yds and Play Result, Conditioned on Opp. Stats
        if playcall == 'pass':
            sim_yds, play_result = self.generate_pass_play(other_team, rand_float)
        elif playcall == 'run':
            sim_yds, play_result = self.generate_run_play(other_team, rand_float)
        else:
            sim_yds, play_result = self.generate_special_teams_play(rand_float)
        
        # Run Play Time Off Game Clock
        self.time_remaining -= self.std_play_duration
        other_team.time_remaining -= other_team.std_play_duration
        
        stat_bunch = (self.name, (self.score, self.opp_score), play_result, round(sim_yds, 1), self.down,
                      round(self.yards_to_first, 1), round(self.yards_to_endzone, 1))
        
        # Append Play Result to Lists
        self.all_plays.append(stat_bunch)
        other_team.all_plays.append(stat_bunch)
        self.off_plays.append(stat_bunch)
        other_team.opp_off_plays.append(stat_bunch)
        
        # Check for Turnover on Downs
        if self.down == 4:
            if playcall in ['pass', 'run']:
                if (sim_yds < self.yards_to_first) and (sim_yds < self.yards_to_endzone):
                    self.possession = False
                    self.run_clock = False
                    self.plays_since_poss = 0
                    self.down = 1
                    self.yards_to_first = 10
                    self.yards_to_endzone -= sim_yds
                    self.yards_to_endzone = 100 - self.yards_to_endzone

                    other_team.possession = True
                    other_team.run_clock = False
                    other_team.plays_since_poss = 0
                    other_team.down = 1
                    other_team.yards_to_first = 10
                    other_team.yards_to_endzone -= sim_yds
                    other_team.yards_to_endzone = 100 - self.yards_to_endzone
                    
                    if playcall == 'pass':
                        self.pass_plays += 1
                        other_team.opp_pass_plays += 1
                        
                        if play_result == 'pass_gain':
                            self.yards += sim_yds
                            self.yds_pass_plays += sim_yds
                            self.completions += 1
                            self.yds_completions += sim_yds
                            other_team.opp_yards += sim_yds
                            other_team.opp_yds_pass_plays += sim_yds
                            other_team.opp_completions += 1
                            other_team.opp_yds_completions += sim_yds
                            
                        elif play_result != 'pass_int':
                            self.yards += sim_yds
                            self.yds_pass_plays += sim_yds
                            other_team.opp_yards += sim_yds
                            other_team.opp_yds_pass_plays += sim_yds
                        
                            
                        
                    elif playcall == 'run':
                        self.run_plays += 1
                        self.yards += sim_yds
                        self.yds_run_plays += sim_yds

                        other_team.opp_run_plays += 1
                        other_team.opp_yards += sim_yds
                        other_team.opp_yds_run_plays += sim_yds
                    
                    return other_team.generate_off_play(other_team=self)
                
                elif sim_yds >= self.yards_to_endzone:
                    return self.touchdown(other_team, playcall, play_result, self.yards_to_endzone)
                
                elif sim_yds >= self.yards_to_first:
                    return self.first_down(other_team, playcall, play_result, sim_yds)
            
            # Punts & FGs
            else:
                self.possession = False
                self.run_clock = False
                self.plays_since_poss = 0
                self.down = 1
                self.yards_to_first = 10
                self.yards_to_endzone -= sim_yds
                self.yards_to_endzone = 100 - self.yards_to_endzone

                other_team.possession = True
                other_team.run_clock = False
                other_team.plays_since_poss = 0
                other_team.down = 1
                other_team.yards_to_first = 10
                other_team.yards_to_endzone -= sim_yds
                other_team.yards_to_endzone = 100 - other_team.yards_to_endzone

                # Punt
                if play_result == 'punt':
                    self.punts += 1
                    other_team.opp_punts += 1

                    # Touchbacks
                    if self.yards_to_endzone >= 100:
                        self.yards_to_endzone = 80
                        other_team.yards_to_endzone = 80

                # Made FG
                elif play_result == 'fg_make':
                    self.score += 3
                    self.yards_to_endzone = 75
                    other_team.opp_score += 3
                    other_team.yards_to_endzone = 75

                return other_team.generate_off_play(other_team=self)
        
        elif self.down <= 3:
            
            if play_result in ['pass_int', 'pass_fum', 'run_fum']:
                self.possession = False
                self.run_clock = False
                self.plays_since_poss = 0
                self.down = 1
                self.yards_to_first = 10
                self.yards_to_endzone = 100 - self.yards_to_endzone

                other_team.possession = True
                other_team.run_clock = False
                other_team.plays_since_poss = 0
                other_team.down = 1
                other_team.yards_to_first = 10
                other_team.yards_to_endzone = 100 - other_team.yards_to_endzone
                
                if play_result == 'pass_int':
                    self.pass_plays += 1
                    self.interceptions += 1
                    other_team.opp_pass_plays += 1
                    other_team.opp_interceptions += 1
                
                elif play_result == 'pass_fum':
                    self.pass_plays += 1
                    self.yards += sim_yds
                    self.yds_pass_plays += sim_yds
                    self.completions += 1
                    self.yds_completions += sim_yds
                    self.fumbles += 1
                    other_team.opp_pass_plays += 1
                    other_team.opp_yards += sim_yds
                    other_team.opp_yds_pass_plays += sim_yds
                    other_team.opp_completions += 1
                    other_team.opp_yds_completions += sim_yds
                    other_team.opp_fumbles += 1
                        
                elif play_result == 'run_fum':
                    self.run_plays += 1
                    self.yards += sim_yds
                    self.yds_run_plays += sim_yds
                    self.fumbles += 1
                    other_team.opp_run_plays += 1
                    other_team.opp_yards += sim_yds
                    other_team.opp_yds_run_plays += sim_yds
                    other_team.opp_fumbles += 1
                
                return other_team.generate_off_play(other_team=self)
            
            # Check for TD
            if sim_yds >= self.yards_to_endzone:
                return self.touchdown(other_team, playcall, play_result, sim_yds)
            
            # Check for 1st Down
            elif sim_yds >= self.yards_to_first:
                return self.first_down(other_team, playcall, play_result, sim_yds)
            
            # All other non-turnover downs
            else:
                self.down += 1
                self.plays_since_poss += 1
                self.yards_to_endzone -= sim_yds
                self.yards_to_first -= sim_yds

                other_team.down += 1
                other_team.plays_since_poss += 1
                other_team.yards_to_endzone -= sim_yds
                other_team.yards_to_first -= sim_yds

                # Check for Incomplete
                if play_result == 'pass_inc':
                    self.possession = True
                    self.run_clock = False
                    self.incompletions += 1
                    self.pass_plays += 1

                    other_team.possession = False
                    other_team.run_clock = False
                    other_team.opp_incompletions += 1
                    other_team.opp_pass_plays += 1

                # All other gains
                else:
                    self.possession = True
                    self.run_clock = True
                    other_team.possession = False
                    other_team.run_clock = True

                    if playcall == 'pass':
                        self.pass_plays += 1
                        self.yards += sim_yds
                        self.yds_pass_plays += sim_yds
                        other_team.opp_pass_plays += 1
                        other_team.opp_yards += sim_yds
                        other_team.opp_yds_pass_plays += sim_yds
                        
                        if play_result == 'pass_gain':
                            self.completions += 1
                            self.yds_completions += sim_yds
                    else:
                        self.run_plays += 1
                        self.yards += sim_yds
                        self.yds_run_plays += sim_yds

                        other_team.opp_run_plays += 1
                        other_team.opp_yards += sim_yds
                        other_team.opp_yds_run_plays += sim_yds

                    self.time_remaining -= self.std_run_clock_duration
                    other_team.time_remaining -= other_team.std_run_clock_duration

                return self.generate_off_play(other_team=other_team)
        
    
    def choose_playcall(self):
        
        if self.down <= 3:
            # Calc playcall factor, which is used to generate run_prob
            if self.yards_to_first >= 11:
                playcall_factor = 3
            elif self.yards_to_first >= 7:
                playcall_factor = 2
            elif self.yards_to_first >= 3:
                playcall_factor = 1
            else:
                playcall_factor = 0

            # Get run_prob from playcall matrix, using playcall factor and down
            down_factor = self.down - 1
            updated_playcall_matrix = self.adjust_playcall_matrix()
            pass_prob = updated_playcall_matrix[playcall_factor][down_factor]

            # Randomly select a weighted playcall given the situation
            if np.random.random() > pass_prob:
                playcall = 'run'
            else:
                playcall = 'pass'
        else:
            if self.yards_to_endzone <= 20:
                if self.yards_to_first <= 1:
                    playcall = 'run'
                elif self.yards_to_first <= 4:
                    playcall = 'pass'
                else:
                    playcall = 'punt/fg'
            elif self.yards_to_endzone <= 60:
                if self.yards_to_first <= 1:
                    playcall = 'run'
                elif self.yards_to_first <= 2:
                    playcall = 'pass'
                else:
                    playcall = 'punt/fg'
            else:
                playcall = 'punt/fg'
            
        return playcall
    
    def adjust_playcall_matrix(self):
        
        # Score Factor
        diff = self.score - self.opp_score
        if diff > 14:
            score_factor = 0.50
        elif diff > 7:
            score_factor = 0.66
        elif diff > 3:
            score_factor = 0.80
        elif diff > 0:
            score_factor = 0.95
        elif diff == 0:
            score_factor = 1.00
        elif diff < 0:
            score_factor = 1.05
        elif diff < -3:
            score_factor = 1.33
        elif diff < -7:
            score_factor = 1.50
        elif diff < -14:
            score_factor = 2.00
        
        # Dist. to Endzone Factor
        if self.yards_to_endzone <= 2:
            endzone_factor = 0.25
        elif self.yards_to_endzone <= 5:
            endzone_factor = 0.33
        elif self.yards_to_endzone <= 10:
            endzone_factor = 0.50
        elif self.yards_to_endzone <= 20:
            endzone_factor = 0.80
        elif self.yards_to_endzone <= 40:
            endzone_factor = 1.00
        else:
            endzone_factor = 1.10
            
        # Time Factor
        used_time = self.game_duration - self.time_remaining
        agg_factor = (used_time ** 2) / ((self.game_duration / 2) ** 2)
        time_aggression_factor = self.min_max_func(agg_factor)
        
        new_matrix = np.ones((self.playcall_matrix.shape[0], self.playcall_matrix.shape[1]))
        for row in range(self.playcall_matrix.shape[0]):
            for col in range(self.playcall_matrix.shape[1]):
                new_val = self.playcall_matrix[row][col] * score_factor * endzone_factor * time_aggression_factor
                new_matrix[row][col] = new_val
                
        return new_matrix / new_matrix.max()
    
    def generate_pass_play(self, other_team, rand_float):
        off_ypc_z = self.off_ypc_z
        def_ypc_z = other_team.def_ypc_z
        net_z = off_ypc_z + def_ypc_z
        ovr_std = self.std_pass_yds_std
        yds_per_comp = net_z * ovr_std + self.league_average_ypc
        
        sim_yds = -1 # default
        while sim_yds <= 0:     
            sim_yds = np.random.normal(mean=yds_per_comp, sigma=ovr_std)

        play_result = 'pass_gain'
        bar = 0
        
        # Checking for Sack
        bar += self.league_average_sack_pct
        if rand_float <= bar:
            sim_yds = np.random.normal(loc=-self.league_average_sack_yds_lost,
                                       scale=self.league_average_sack_std)
            play_result = 'sack'
        else:
            
            # Checking for INT
            bar += self.ovr_int_pct
            if rand_float <= bar:
                sim_yds -= self.league_average_int_yds_lost
                play_result = 'pass_int'
            else:

                # Checking for incomplete pass
                bar += self.real_inc_pct
                if rand_float <= bar:
                    sim_yds = 0
                    play_result = 'pass_inc'
                else:

                    # Checking for fumble
                    bar += self.league_average_lost_fumble_pct
                    if rand_float <= bar:
                        play_result = 'pass_fum'
                        
        return sim_yds, play_result
    
    def generate_run_play(self, other_team, rand_float):
        def_ypr_z = other_team.def_ypr_z
        net_z = self.off_ypr_z + def_ypr_z
        exp_yds = np.random.lognormal(mean=net_z-5.0, sigma=2.0)
        sim_yds = exp_yds * self.std_run_yds_std + self.league_average_ypr
#         ovr_std = self.std_run_yds_std # * ovr_std_factor
        
        play_result = 'run_gain'

#         # Big TD runs
#         if sim_yds >= 20:
#             sim_yds = self.yards_to_endzone if np.random.random() > 0.67 else sim_yds
        
        if rand_float <= self.league_average_lost_fumble_pct:
            play_result = 'run_fum'
        
        return sim_yds, play_result
    
    def generate_special_teams_play(self, rand_float):
        sim_yds = 0
        
        # FG under 52 yds
        if self.yards_to_endzone <= 15:
            fg_pct = 98
            play_result = 'fg_make' if rand_float <= fg_pct else 'fg_miss'
            sim_yds = -7 if play_result == 'fg_miss' else sim_yds
        elif self.yards_to_endzone <= 25:
            fg_pct = 85
            play_result = 'fg_make' if rand_float <= fg_pct else 'fg_miss'
            sim_yds = -7 if play_result == 'fg_miss' else sim_yds
        elif self.yards_to_endzone <= 35:
            fg_pct = 70
            play_result = 'fg_make' if rand_float <= fg_pct else 'fg_miss'
            sim_yds = -7 if play_result == 'fg_miss' else sim_yds
            
        # Punt
        else:
            play_result = 'punt'
            sim_yds = np.random.normal(loc=self.league_average_net_punt_yds, scale=5)
        
        return sim_yds, play_result
    
    def touchdown(self, other_team, playcall, play_result, sim_yds):
        self.score += 7
        self.possession = False
        self.run_clock = False
        self.plays_since_poss = 0
        self.down = 1
        self.yards_to_first = 10
        self.yards_to_endzone = 75

        other_team.opp_score += 7
        other_team.possession = True
        other_team.run_clock = False
        other_team.plays_since_poss = 0
        other_team.down = 1
        other_team.yards_to_first = 10
        other_team.yards_to_endzone = 75

        if playcall == 'pass':
            self.pass_plays += 1
            self.yards += sim_yds
            self.yds_pass_plays += sim_yds
            other_team.opp_pass_plays += 1
            other_team.opp_yards += sim_yds
            other_team.opp_yds_pass_plays += sim_yds
            
            if play_result == 'pass_gain':
                self.completions += 1
                self.yds_completions += sim_yds
                other_team.opp_completions += 1
                other_team.opp_yds_completions += sim_yds

        elif playcall == 'run':
            self.run_plays += 1
            self.yards += sim_yds
            self.yds_run_plays += sim_yds
            self.fumbles += 1
            other_team.opp_run_plays += 1
            other_team.opp_yards += sim_yds
            other_team.opp_yds_run_plays += sim_yds
            other_team.opp_fumbles += 1

        return other_team.generate_off_play(other_team=self)
    
    def first_down(self, other_team, playcall, play_result, sim_yds):
        self.possession = True
        self.run_clock = True
        self.plays_since_poss += 1
        self.down = 1
        self.yards_to_first = 10
        self.yards_to_endzone -= sim_yds

        other_team.possession = False
        other_team.run_clock = True
        other_team.plays_since_poss += 1
        other_team.down = 1
        other_team.yards_to_first = 10
        other_team.yards_to_endzone -= sim_yds


        if playcall == 'pass':
            self.pass_plays += 1
            self.yards += sim_yds
            self.yds_pass_plays += sim_yds
            other_team.opp_pass_plays += 1
            other_team.opp_yards += sim_yds
            other_team.opp_yds_pass_plays += sim_yds
            
            if play_result == 'pass_gain':
                self.completions += 1
                self.yds_completions += sim_yds
                other_team.opp_completions += 1
                other_team.opp_yds_completions += sim_yds

        elif playcall == 'run':
            self.run_plays += 1
            self.yards += sim_yds
            self.yds_run_plays += sim_yds
            self.fumbles += 1
            other_team.opp_run_plays += 1
            other_team.opp_yards += sim_yds
            other_team.opp_yds_run_plays += sim_yds
            other_team.opp_fumbles += 1

        self.time_remaining -= self.std_run_clock_duration
        other_team.time_remaining -= self.std_run_clock_duration
        
        return self.generate_off_play(other_team=other_team)
    
    def apply_lr_step(self, other_team, lr=None):
        return None
#         lr = lr if lr else self.lr
        
#         ypp_mean, ypr_mean = self.league_average_ypc, self.league_average_ypr
#         ypp_std, ypr_std = self.league_average_ypc_std, self.league_average_ypr_std
        
#         # Calculate Step Size = Learning Rate x Z_Score
#         ypp_score = self.calc_z_score(self.game_ypp, ypp_mean, ypp_std) - other_team.def_ypc_z
#         ypr_score = self.calc_z_score(self.game_ypr, ypr_mean, ypr_std) - other_team.def_ypr_z
#         def_ypc_score = self.calc_z_score(self.game_def_ypc, ypp_mean, ypp_std) - other_team.off_ypc
#         def_ypr_score = self.calc_z_score(self.game_def_ypr, ypr_mean, ypr_std) - other_team.off_ypr_z
        
#         # Add step to base stats of both teams
#         self.off_ypc += (ypp_score * lr)
#         self.off_ypr_z += (ypr_score * lr)
#         self.def_ypc_z += (def_ypc_score * lr)
#         self.def_ypr_z += (def_ypr_score * lr)
        
#         if other_team.lr:          
#             oypp_score = self.calc_z_score(other_team.game_ypp, ypp_mean, ypp_std) - self.def_ypc_z
#             oypr_score = self.calc_z_score(other_team.game_ypr, ypr_mean, ypr_std) - self.def_ypr_z
#             odef_ypc_score = self.calc_z_score(other_team.game_def_ypc, ypp_mean, ypp_std) - self.off_ypc
#             odef_ypr_score = self.calc_z_score(other_team.game_def_ypr, ypr_mean, ypr_std) - self.def_ypr_z
            
#             other_team.off_ypc += (oypp_score * lr)
#             other_team.off_ypr_z += (oypr_score * lr)
#             other_team.def_ypc_z += (odef_ypc_score * lr)
#             other_team.def_ypr_z += (odef_ypr_score * lr)
            
    def set_stats_to_defaults(self, other_team=None):
        self.game_on = True
        self.possession = True
        self.run_clock = True
        self.plays_since_poss = 0
        self.down = 1
        self.yards_to_first = 10
        self.yards_to_endzone = 75
        self.score = 0
        self.opp_score = 0
        self.time_remaining = 3600
        self.all_plays = []
        
        # Record/results
        
        self.off_plays = []
        self.drives = 0
        self.yards = 0
        self.pass_plays = 0
        self.yds_pass_plays = 0
        self.sacks = 0
        self.run_plays = 0
        self.yds_run_plays = 0
        self.completions = 0
        self.yds_completions = 0
        self.incompletions = 0
        self.interceptions = 0
        self.fumbles = 0
        self.punts = 0
        
        self.opp_off_plays = []
        self.opp_drives = 0
        self.opp_yards = 0
        self.opp_pass_plays = 0
        self.opp_yds_pass_plays = 0
        self.opp_sacks = 0
        self.opp_run_plays = 0
        self.opp_yds_run_plays = 0
        self.opp_completions = 0
        self.opp_yds_completions = 0
        self.opp_incompletions = 0
        self.opp_interceptions = 0
        self.opp_fumbles = 0
        self.opp_punts = 0
        
        if other_team:
            other_team.set_stats_to_defaults()
            
    @staticmethod
    def calc_z_score(stat, mean, std):
        return (stat - mean) / std
            
    @staticmethod
    def generate_playcall_matrix(yd_factors, down_factors, run_pass_bias=None):
        run_pass_factor = 0.25
        base_array = np.ones((len(yd_factors), len(down_factors)))
        for i, val_1 in enumerate(yd_factors):
            for j, val_2 in enumerate(down_factors):
                base_array[i][j] = np.mean([val_1, val_2])
                
        min_val, max_val = base_array.min(), base_array.max()
        playcall_matrix = base_array - min_val
        playcall_matrix = playcall_matrix / max_val
        playcall_matrix = 1 - playcall_matrix

        for i in range(playcall_matrix.shape[0]):
            for j in range(playcall_matrix.shape[1]):
                curr_val = playcall_matrix[i][j]
                if run_pass_bias == 'pass':
                    diff = 1 - curr_val
                    new_add = diff * run_pass_factor
                    new_val = curr_val + new_add
                    playcall_matrix[i][j] = new_val
                elif run_pass_bias == 'run':
                    diff = curr_val
                    new_sub = diff * run_pass_factor
                    new_val = curr_val - new_sub
                    playcall_matrix[i][j] = new_val
                    
        return playcall_matrix
    
    @staticmethod
    def generate_random_yd_and_down_factors(base_yd_factors=[0.95, 0.66, 0.5, 0.25],
                                            down_factors=[0.95, 0.5, 0.2]):
        base_yd_factors, down_factors = np.array(base_yd_factors), np.array(down_factors)
        base_yd_stds = base_yd_factors * 0.10
        down_stds = down_factors * 0.10

        sample_yd_factors = []
        sample_down_factors = []
        prev_elem = np.inf
        for i in range(len(base_yd_factors)):
            rand_sample = np.random.normal(loc=base_yd_factors[i], scale=base_yd_stds[i])
            while (rand_sample < 0) or (rand_sample > 1) or (rand_sample > prev_elem):
                rand_sample = np.random.normal(loc=base_yd_factors[i], scale=base_yd_stds[i])
            prev_elem = rand_sample
            sample_yd_factors.append(round(rand_sample, 2))

        prev_elem = np.inf
        for i in range(len(down_factors)):
            rand_sample = np.random.normal(loc=down_factors[i], scale=down_stds[i])
            while (rand_sample < 0) or (rand_sample > 1) or (rand_sample > prev_elem):
                rand_sample = np.random.normal(loc=down_factors[i], scale=down_stds[i])
            prev_elem = rand_sample
            sample_down_factors.append(round(rand_sample, 2))

        return sample_yd_factors, sample_down_factors

    
    @staticmethod
    def sim_season(n_games, team_a, team_b):
    
        team_a_wins, team_b_wins, draws = 0, 0, 0
        a_drives, b_drives = 0, 0
        a_plays, b_plays = 0, 0
        a_pass_plays, b_pass_plays = 0, 0
        a_comps, b_comps = 0, 0
        a_run_plays, b_run_plays = 0, 0
        team_a_pf, team_b_pf = 0, 0
        a_tot_margin, b_tot_margin = 0, 0
        a_ypp, b_ypp = 0, 0
        a_ypr, b_ypr = 0, 0
        a_yds_c, b_yds_c = 0, 0

        default_val_dict = {
                        'a_off_ypc'        : Team.league_average_ypc,
                        'b_off_ypc'        : Team.league_average_ypc,
                        'a_def_ypc'        : Team.league_average_ypc,
                        'b_def_ypc'        : Team.league_average_ypc,
                        'a_off_ypr'        : Team.league_average_ypr,
                        'b_off_ypr'        : Team.league_average_ypr,
                        'a_def_ypr'        : Team.league_average_ypr,
                        'b_def_ypr'        : Team.league_average_ypr,
                        'a_comp_pct'       : Team.league_average_comp_pct,
                        'b_comp_pct'       : Team.league_average_comp_pct,
                        'a_pnt_ypp'        : Team.league_average_net_punt_yds,
                        'b_pnt_ypp'        : Team.league_average_net_punt_yds,
                        'a_yd_factors'     : Team.league_average_yard_factors,
                        'b_yd_factors'     : Team.league_average_yard_factors,
                        'a_down_factors'   : Team.league_average_down_factors,
                        'b_down_factors'   : Team.league_average_down_factors
                        }

        # Loop through n_games, and tally the stats
        for _ in range(n_games):

            # Draw random int: 0 or 1 to decide which team starts with poss.
            rand_int = np.random.randint(0, 2)

            # Team A starts with poss.
            if rand_int == 0:
                team_b.possession = False
                team_a.possession = True
                winner, name, *teams = team_a.play_opponent(team_b)

                if name == 'A':
                    team_a, team_b = teams[0], teams[1]
                else:
                    team_a, team_b = teams[1], teams[0]
                    
                if winner == 'A':
                    team_a_wins += 1
                elif winner == 'd':
                    draws += 1
                else:
                    team_b_wins += 1

            # Team B starts with poss.
            elif rand_int == 1:
                team_a.possession = False
                team_b.possession = True
                winner, name, *teams = team_b.play_opponent(team_a)

                if name == 'A':
                    team_a, team_b = teams[0], teams[1]
                else:
                    team_a, team_b = teams[1], teams[0]
                
                if winner == 'A':
                    team_a_wins += 1
                elif winner == 'd':
                    draws += 1
                else:
                    team_b_wins += 1

            a_plys = team_a.pass_plays + team_a.run_plays
            b_plys = team_b.pass_plays + team_b.run_plays
            a_diff = np.sum(team_a.diff_tot)
            b_diff = np.sum(team_b.diff_tot)


            team_a_pf += team_a.score
            team_b_pf += team_b.score
            a_drives += team_a.drives
            b_drives += team_b.drives
            a_plays += a_plys
            b_plays += b_plys
            a_pass_plays += team_a.pass_plays
            b_pass_plays += team_b.pass_plays
            a_run_plays += team_a.run_plays
            b_run_plays += team_b.run_plays
            a_tot_margin += a_diff
            b_tot_margin += b_diff
            a_ypp += team_a.game_ypp
            b_ypp += team_b.game_ypp
            a_ypr += team_a.game_ypr
            b_ypr += team_b.game_ypr
            a_comps += team_a.completions
            b_comps += team_b.completions
            a_yds_c += team_a.yds_completions
            b_yds_c += team_b.yds_completions
            
        bunch = (
                team_a,
                team_b,
                team_a_wins,
                team_b_wins,
                draws,
                team_a_pf,
                team_b_pf,
                a_drives,
                b_drives,
                a_plays,
                b_plays,
                a_pass_plays,
                b_pass_plays,
                a_run_plays,
                b_run_plays,
                a_tot_margin,
                b_tot_margin,
                a_ypp,
                b_ypp,
                a_ypr,
                b_ypr,
                a_comps,
                b_comps,
                a_yds_c,
                b_yds_c
                )

        return bunch

### Run Sims

In [60]:
a_off_ypc, a_def_ypc = 11.1, 11.1
b_off_ypc, b_def_ypc = 11.1, 11.1
a_comp_pct = 0.70
b_comp_pct = 0.68
a_yd_factors = [0.95, 0.67, 0.5, 0.25]
a_down_factors = [0.95, 0.5, 0.2]
b_yd_factors = [0.95, 0.67, 0.5, 0.25]
b_down_factors = [0.95, 0.5, 0.2]
a_lr, b_lr = 0.01, 0.01

team_a = Team('A', learning=False, off_ypc=a_off_ypc, def_ypc=a_def_ypc, comp_pct=a_comp_pct,
              yd_factors=a_yd_factors, down_factors=a_down_factors)
team_b = Team('B', learning=False, off_ypc=b_off_ypc, def_ypc=b_def_ypc, comp_pct=b_comp_pct,
              yd_factors=b_yd_factors, down_factors=b_down_factors)

team_b.possession = False
team_a.possession = True
winner, name, *teams = team_a.play_opponent(team_b)
if name == 'A':
    team_a, team_b = teams[0], teams[1]
else:
    team_a, team_b = teams[1], teams[0]
a_score, b_score = team_a.score, team_b.score
a_drives, b_drives = team_a.drives, team_b.drives
a_plays = team_a.pass_plays + team_a.run_plays
b_plays = team_b.pass_plays + team_b.run_plays
a_diff = np.sum(team_a.diff_tot) / a_plays
b_diff = np.sum(team_b.diff_tot) / b_plays
win_score = a_score if a_score >= b_score else b_score
lose_score = b_score if b_score < a_score else a_score
winner.name, win_score, lose_score, a_drives, a_plays, team_a.pass_plays, team_a.run_plays, a_diff, b_diff

('A', 31, 21, 8, 57, 31, 26, 8.192982456140351, -15.446428571428571)

In [43]:
yd_factors, down_factors = Team.generate_random_yd_and_down_factors()
base_playcall_matrix = Team.generate_playcall_matrix(yd_factors, down_factors, run_pass_bias=None)
pass_playcall_matrix = Team.generate_playcall_matrix(yd_factors, down_factors, run_pass_bias='pass')
print(sum(sum(pass_playcall_matrix - base_playcall_matrix)))
base_playcall_matrix

1.1965317919075145


array([[0.22543353, 0.45086705, 0.6416185 ],
       [0.35260116, 0.57803468, 0.76878613],
       [0.38728324, 0.61271676, 0.80346821],
       [0.58381503, 0.80924855, 1.        ]])

### Generate League of N-Teams

In [50]:
import string

def generate_teams(team_obj, lr, n_teams=32, **kwargs):
    
    # Build name lookup dict
    names = list(string.ascii_uppercase)
    add_list = ['Aa', 'Bb', 'Cc', 'Dd', 'Ee', 'Ff', 'Gg']
    names.extend(add_list)
    team_name_dict = dict(zip(range(n_teams+1), names))

    team_list = [team_obj]
    for i in range(1, n_teams):
        team_name = team_name_dict[i]
        
        nrm = lambda loc, scale: np.random.normal(loc=loc, scale=scale)
        sample_off_ypp = nrm(loc=team_obj.league_average_ypp, scale=team_obj.league_average_ypp_std)
        sample_def_ypp = nrm(loc=team_obj.league_average_ypp, scale=team_obj.league_average_ypp_std)
        sample_off_ypr = nrm(loc=team_obj.league_average_ypr, scale=team_obj.league_average_ypr_std)
        sample_def_ypr = nrm(loc=team_obj.league_average_ypr, scale=team_obj.league_average_ypr_std)
        sample_comp_pct = nrm(loc=team_obj.league_average_comp_pct, scale=team_obj.league_average_comp_pct_std)
        sample_punt_ypp = nrm(loc=team_obj.league_average_net_punt_yds, scale=team_obj.league_average_punt_std)
        
        yd_factors, down_factors = generate_random_yd_and_down_factors(**kwargs)
        
        rnd_int = np.random.randint(3)
        if rnd_int == 0:
            run_pass_bias = 'run'
        elif rnd_int == 1:
            run_pass_bias = 'pass'
        else:
            run_pass_bias = None
        
        Team.base_lr = lr if lr else Team.base_lr
        team = Team(name=team_name, learning=True, off_ypp=sample_off_ypp, def_ypp=sample_def_ypp,
                    off_ypr=sample_off_ypr, def_ypr=sample_def_ypr, comp_pct=sample_comp_pct,
                    pnt_ypp=sample_punt_ypp, yd_factors=yd_factors, down_factors=down_factors,
                    run_pass_bias=run_pass_bias)
        team_list.append(team)
    
    name_obj_dict = dict(zip(names, team_list))
    
    return team_list, team_name_dict, name_obj_dict

### Simulate Full Season

In [51]:
team_a = Team('A', learning=False, off_ypc=11.1, off_ypr=4.3, def_ypc=11.1, def_ypr=4.3)
team_b = Team('B', learning=False, off_ypc=11.1, off_ypr=4.3, def_ypc=11.1, def_ypr=4.3)
n_games = 1000

team_a, team_b, team_a_wins, team_b_wins, draws, *bunch = Team.sim_season(n_games, team_a, team_b)
*tmp_bunch, a_ypr, b_ypr, a_comps, b_comps, a_yds_c, b_yds_c = bunch

tot_games = team_a_wins + team_b_wins + draws
a_ypr_pg = a_ypr / tot_games
b_ypr_pg = b_ypr / tot_games
a_ypc = a_yds_c / a_comps
b_ypc = b_yds_c / b_comps
ypc_diff_new = (team_a.off_ypc_z - team_a.init_off_ypc_z) * team_a.league_average_ypc_std
ypr_diff_new = (team_a.off_ypr_z - team_a.init_off_ypr_z) * team_a.league_average_ypr_std
oypc_diff_new = (team_a.def_ypc_z - team_a.init_def_ypc_z) * team_a.league_average_ypc_std
oypr_diff_new = (team_a.def_ypr_z - team_a.init_def_ypr_z) * team_a.league_average_ypr_std

rnd = lambda x: round(x, 2)
total_change = abs(ypc_diff_new) + abs(ypr_diff_new) + abs(oypc_diff_new) + abs(oypr_diff_new)
print(a_ypc, a_ypr_pg, b_ypc, b_ypr_pg)
rnd(total_change), rnd(ypc_diff_new), rnd(ypr_diff_new), rnd(oypc_diff_new), rnd(oypr_diff_new)

12.653900198966632 5.096225526549665 12.638433782251507 5.105307533444598


(0.0, 0.0, 0.0, 0.0, 0.0)

In [2280]:
team_a = Team('A', learning=False, off_ypp=11.1, off_ypr=4.3, def_ypp=11.1, def_ypr=4.3)
team_b = Team('B', learning=False, off_ypp=11.1, off_ypr=4.3, def_ypp=11.1, def_ypr=4.3)
n_games = 10000

team_a, team_b, team_a_wins, team_b_wins, draws, *bunch = Team.sim_season(n_games, team_a, team_b)
*tmp_bunch, a_ypp, b_ypp, a_ypr, b_ypr = bunch

tot_games = team_a_wins + team_b_wins + draws
a_ypp_pg = a_ypp / tot_games
a_ypr_pg = a_ypr / tot_games
b_ypp_pg = b_ypp / tot_games
b_ypr_pg = b_ypr / tot_games
ypp_diff_new = (team_a.off_ypp_z - team_a.init_off_ypp_z) * team_a.league_average_ypp_std
ypr_diff_new = (team_a.off_ypr_z - team_a.init_off_ypr_z) * team_a.league_average_ypr_std
oypp_diff_new = (team_a.def_ypp_z - team_a.init_def_ypp_z) * team_a.league_average_ypp_std
oypr_diff_new = (team_a.def_ypr_z - team_a.init_def_ypp_z) * team_a.league_average_ypr_std

rnd = lambda x: round(x, 2)
total_change = abs(ypp_diff_new) + abs(ypr_diff_new) + abs(oypp_diff_new) + abs(oypr_diff_new)
print(a_ypp_pg, a_ypr_pg, b_ypp_pg, b_ypr_pg)
rnd(total_change), rnd(ypp_diff_new), rnd(ypr_diff_new), rnd(oypp_diff_new), rnd(oypr_diff_new)

7.2316224230478054 4.249306883954852 7.227472115404866 4.248706695366249


(0.0, 0.0, 0.0, 0.0, 0.0)

In [2629]:
def simulate_full_league_season(n_sims, n_games_per_sim, generate_teams, subject_team, learning=True, lr=0.001):
    
    gen_team_list, team_name_dict, name_obj_dict = generate_teams(subject_team, lr=lr, n_teams=32)
    name_obj_dict[subject_team.name] = subject_team
    zero_list = list(np.zeros(10))
    season_results = dict.fromkeys(name_obj_dict, zero_list)
    
    for i in range(n_sims):
        
        if i % 250 == 0:
            print(f'Simmed {i} games so far...')
        
        # Select two unique numbers
        rand_int = np.random.randint(32)
        rand_int_2 = np.random.randint(32)
        while rand_int == rand_int_2:
            rand_int_2 = np.random.randint(32)
            
        # Fetch names and results
        team_x_name = team_name_dict[rand_int]
        team_y_name = team_name_dict[rand_int_2]
        team_x_results = season_results[team_x_name]
        team_y_results = season_results[team_y_name]
        
        # Objects themselves
        team_x = name_obj_dict[team_x_name]
        team_y = name_obj_dict[team_y_name]
        
        # X Stats
        x_off_ypp, x_def_ypp = team_x.off_ypp, team_x.def_ypp
        x_off_ypr, x_def_ypr = team_x.off_ypr, team_x.def_ypr
        x_comp_pct, x_pnt_ypp = team_x.real_comp_pct, team_x.pnt_ypp
        x_yd_factors, x_down_factors = team_x.yd_factors, team_x.down_factors

        # Y Stats
        y_off_ypp, y_def_ypp = team_y.off_ypp, team_y.def_ypp
        y_off_ypr, y_def_ypr = team_y.off_ypr, team_y.def_ypr
        y_comp_pct, y_pnt_ypp = team_y.real_comp_pct, team_y.pnt_ypp
        y_yd_factors, y_down_factors = team_y.yd_factors, team_y.down_factors
        
        # Bunch data
        bunch = sim_season(n_games_per_sim, team_x, team_y)
        
        # Parse and re-arrange
        team_x, team_y, team_x_wins, team_y_wins, draws, team_x_pf, team_y_pf, *first_bunch = bunch
        x_drives, y_drives, x_plays, y_plays, x_pass_plays, y_pass_plays, *second_bunch = first_bunch
        x_run_plays, y_run_plays, x_margin, y_margin = second_bunch
        
        x_bunch = [team_x_wins, team_y_wins, draws, team_x_pf, team_y_pf, x_drives, x_plays,
                   x_pass_plays, x_run_plays, x_margin]
        y_bunch = [team_y_wins, team_x_wins, draws, team_y_pf, team_x_pf, y_drives, y_plays,
                   y_pass_plays, y_run_plays, y_margin]
        
        updated_x_results = np.array(team_x_results) + np.array(x_bunch)
        updated_y_results = np.array(team_y_results) + np.array(y_bunch)
        
        season_results[team_x_name] = updated_x_results
        season_results[team_y_name] = updated_y_results
            
        name_obj_dict[team_x.name] = team_x
        name_obj_dict[team_y.name] = team_y
        
    return (season_results, name_obj_dict)

### Generate Subject Team's Stats

In [2630]:
import time

off_ypp, def_ypp = 7.2, 7.2
off_ypr, def_ypr = 4.3, 4.3
comp_pct = 0.65
pnt_ypp = 38
yd_factors = [0.95, 0.67, 0.5, 0.25]
down_factors = [0.95, 0.5, 0.2]
run_pass_bias = None
learning = False
lr = 0.001

n_sims = 10000
n_games_per_sim = 1

subject_team = Team('A', learning=learning, off_ypp=off_ypp, def_ypp=def_ypp, off_ypr=off_ypr, def_ypr=def_ypr,
                    comp_pct=comp_pct, pnt_ypp=pnt_ypp, yd_factors=yd_factors, down_factors=down_factors,
                    run_pass_bias=run_pass_bias)

start = time.time()
bunch = simulate_full_league_season(n_sims, n_games_per_sim, generate_teams, subject_team, learning=learning,
                                    lr=lr)
season_results, name_obj_dict = bunch
end = time.time()
print(end-start)

Simmed 0 games so far...
Simmed 250 games so far...
Simmed 500 games so far...
Simmed 750 games so far...
Simmed 1000 games so far...
Simmed 1250 games so far...
Simmed 1500 games so far...
Simmed 1750 games so far...
Simmed 2000 games so far...
Simmed 2250 games so far...
Simmed 2500 games so far...
Simmed 2750 games so far...
Simmed 3000 games so far...
Simmed 3250 games so far...
Simmed 3500 games so far...
Simmed 3750 games so far...
Simmed 4000 games so far...
Simmed 4250 games so far...
Simmed 4500 games so far...
Simmed 4750 games so far...
Simmed 5000 games so far...
Simmed 5250 games so far...
Simmed 5500 games so far...
Simmed 5750 games so far...
Simmed 6000 games so far...
Simmed 6250 games so far...
Simmed 6500 games so far...
Simmed 6750 games so far...
Simmed 7000 games so far...
Simmed 7250 games so far...
Simmed 7500 games so far...
Simmed 7750 games so far...
Simmed 8000 games so far...
Simmed 8250 games so far...
Simmed 8500 games so far...
Simmed 8750 games so far..

In [2631]:
main_df = pd.DataFrame.from_dict(season_results, orient='index', columns=['wins', 'losses', 'draws', 'pf', 'pa',
                                                                          'drives', 'plays', 'pass_plays',
                                                                          'run_plays', 'avg_margin'])

main_df.reset_index(drop=False, inplace=True)

main_df['win_pct'] = main_df['wins'] / (main_df['wins'] + main_df['losses'])
main_df['games'] = main_df['wins'] + main_df['losses'] + main_df['draws']
main_df['pf_pg'] = main_df['pf'] / main_df['games']
main_df['pa_pg'] = main_df['pa'] / main_df['games']
main_df['pts_diff'] = main_df['pf_pg'] - main_df['pa_pg']
main_df['dr_pg'] = main_df['drives'] / main_df['games']
main_df['run_plays_pg'] = main_df['run_plays'] / main_df['games']
main_df['pass_pct'] = main_df['pass_plays'] / (main_df['pass_plays'] + main_df['run_plays'])
main_df['avg_margin'] = main_df['avg_margin'] / main_df['plays']

main_df.drop(['pf', 'pa', 'drives', 'plays', 'pass_plays', 'run_plays'], axis=1, inplace=True)
main_df.set_index('index', inplace=True)

### Add to DF a "Pass_Score" Column (Total Val of Each Playcall Matrix) 

In [2632]:
sub_obj = name_obj_dict['A']
off_ypp_diff = sub_obj.off_ypp_z - sub_obj.init_off_ypp_z
off_ypr_diff = sub_obj.off_ypr_z - sub_obj.init_off_ypr_z
def_ypp_diff = sub_obj.def_ypp_z - sub_obj.init_def_ypp_z
def_ypr_diff = sub_obj.def_ypr_z - sub_obj.init_def_ypr_z
off_ypp_diff, off_ypr_diff, def_ypp_diff, def_ypp_diff

(0.0, 0.0, 0.0, 0.0)

In [2633]:
quick_dict = {}
quick_dict_2 = {}
quick_dict_3 = {}
for obj in name_obj_dict.values():
    quick_dict[obj.name] = sum(obj.playcall_matrix)
    quick_dict_2[obj.name] = (obj.off_ypp_z, obj.off_ypr_z, obj.def_ypp_z, obj.def_ypr_z)
    quick_dict_3[obj.name] = (obj.init_off_ypp_z, obj.init_off_ypr_z, obj.init_def_ypp_z, obj.init_def_ypr_z)
s1 = main_df.index.map(quick_dict)
s2 = main_df.index.map(quick_dict_2)
s3 = main_df.index.map(quick_dict_3)
main_df['flat_scores'] = s1
main_df['flat_scores_2'] = s2
main_df['flat_scores_3'] = s3
score_list = main_df['flat_scores'].tolist()
score_list_2 = main_df['flat_scores_2'].tolist()
score_list_3 = main_df['flat_scores_3'].tolist()
main_df.drop(['flat_scores', 'flat_scores_2', 'flat_scores_3'], axis=1, inplace=True)
col_names = ['pass_1st', 'pass_2nd', 'pass_3rd']
main_df[col_names] = pd.DataFrame(score_list, index=main_df.index)
col_names_2 = ['off_ypp', 'off_ypr', 'def_ypp', 'def_ypr']
main_df[col_names_2] = pd.DataFrame(score_list_2, index=main_df.index)
col_names_3 = ['in_off_ypp', 'in_off_ypr', 'in_def_ypp', 'in_def_ypr']
main_df[col_names_3] = pd.DataFrame(score_list_3, index=main_df.index)
main_df.head()

,wins,losses,draws,avg_margin,win_pct,games,pf_pg,pa_pg,pts_diff,dr_pg,...,pass_2nd,pass_3rd,off_ypp,off_ypr,def_ypp,def_ypr,in_off_ypp,in_off_ypr,in_def_ypp,in_def_ypr
index,,,,,,,,,,,,,,,,,,,,,
A,295.0,298.0,31.0,-1.898945,0.497470,624.0,25.041667,25.676282,-0.634615,10.498397,...,2.647368,3.278947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B,258.0,279.0,38.0,-1.281347,0.480447,575.0,26.573913,26.393043,0.180870,10.396522,...,2.242775,3.075145,2.833218,1.104594,1.079001,0.640150,1.590652,0.617232,0.290937,0.351420
C,316.0,272.0,35.0,-1.232696,0.537415,623.0,26.648475,26.157303,0.491172,10.377207,...,2.724324,3.264865,1.670553,0.531983,1.306535,-1.630512,0.386929,0.301253,0.306114,-0.864184
D,291.0,336.0,20.0,-1.284255,0.464115,647.0,27.944359,27.313756,0.630603,10.358578,...,2.026596,2.457447,4.246548,1.589792,0.267533,0.044944,2.021725,0.922693,-0.412372,-0.129622
E,326.0,270.0,24.0,-1.671836,0.546980,620.0,25.161290,25.977419,-0.816129,10.625806,...,2.088983,2.461864,0.752539,1.940718,-0.327633,-0.148903,0.078295,1.131807,-0.774889,-0.086968


In [2634]:
diff_df = main_df.iloc[:, -8:]
diff_df['off_ypp_diff'] = diff_df.loc[:, 'off_ypp'] - diff_df.loc[:, 'in_off_ypp']
diff_df['off_ypr_diff'] = diff_df.loc[:, 'off_ypr'] - diff_df.loc[:, 'in_off_ypr']
diff_df['def_ypp_diff'] = diff_df.loc[:, 'def_ypp'] - diff_df.loc[:, 'in_def_ypp']
diff_df['def_ypr_diff'] = diff_df.loc[:, 'def_ypr'] - diff_df.loc[:, 'in_def_ypr']
new_df = diff_df[['off_ypp_diff', 'off_ypr_diff', 'def_ypp_diff', 'def_ypr_diff']]
new_df.head()

,off_ypp_diff,off_ypr_diff,def_ypp_diff,def_ypr_diff
index,,,,
A,0.000000,0.000000,0.000000,0.000000
B,1.242565,0.487363,0.788064,0.288730
C,1.283623,0.230730,1.000421,-0.766327
D,2.224823,0.667100,0.679905,0.174566
E,0.674244,0.808911,0.447257,-0.061935


In [2635]:
new_cols = ['off_ypp', 'in_off_ypp', 'off_ypr', 'in_off_ypr', 'def_ypp', 'in_def_ypp',  'def_ypr', 'in_def_ypr']
bunch = main_df[new_cols].agg(['mean', 'std'])
bunch

,off_ypp,in_off_ypp,off_ypr,in_off_ypr,def_ypp,in_def_ypp,def_ypr,in_def_ypr
mean,0.706563,-0.060511,-0.244114,-0.102193,0.681893,-0.066850,-0.344098,-0.160620
std,1.511634,1.080390,1.889039,0.990970,1.176157,0.847507,1.694496,0.907815


In [2623]:
new_cols = ['off_ypp', 'in_off_ypp', 'off_ypr', 'in_off_ypr', 'def_ypp', 'in_def_ypp',  'def_ypr', 'in_def_ypr']
bunch = main_df[new_cols].agg(['mean', 'std'])
bunch

,off_ypp,in_off_ypp,off_ypr,in_off_ypr,def_ypp,in_def_ypp,def_ypr,in_def_ypr
mean,-0.075900,-0.163119,0.147382,0.096246,-0.193218,-0.228620,-0.345446,-0.256439
std,1.218404,0.776160,2.012876,1.075233,1.568372,1.048129,1.811652,0.941413


In [2624]:
tot_sum = 0
col_sums = []
for col in new_df.columns:
    summed = new_df[col].abs().sum()
    tot_sum += summed
    col_sums.append(summed)
tot_sum, *col_sums

(70.02059550581544,
 11.688236014270775,
 23.581741207035464,
 13.630775908150495,
 21.119842376358704)

In [2227]:
tot_sum = 0
col_sums = []
for col in new_df.columns:
    summed = new_df[col].abs().sum()
    tot_sum += summed
    col_sums.append(summed)
tot_sum, *col_sums

(23.381290597384606,
 5.245790483877489,
 6.96437137809938,
 5.289545249086805,
 5.881583486320934)

In [2221]:
tot_sum = 0
col_sums = []
for col in new_df.columns:
    summed = new_df[col].abs().sum()
    tot_sum += summed
    col_sums.append(summed)
tot_sum, *col_sums

(24.325961497343616,
 5.273526269770935,
 7.40101297240715,
 5.855833079198877,
 5.795589175966653)

In [2215]:
tot_sum = 0
col_sums = []
for col in new_df.columns:
    summed = new_df[col].abs().sum()
    tot_sum += summed
    col_sums.append(summed)
tot_sum, *col_sums

(20.472358011476746,
 4.84813515724879,
 5.3573550097482885,
 4.547332663690498,
 5.719535180789169)

In [2209]:
tot_sum = 0
col_sums = []
for col in new_df.columns:
    summed = new_df[col].abs().sum()
    tot_sum += summed
    col_sums.append(summed)
tot_sum, *col_sums

(19.674016773465045,
 4.779685254715651,
 4.86247571382633,
 3.810041584642857,
 6.221814220280207)

In [2197]:
main_df.sort_values('pts_diff', ascending=False)

,wins,losses,draws,avg_margin,win_pct,games,pf_pg,pa_pg,pts_diff,dr_pg,...,pass_2nd,pass_3rd,off_ypp,off_ypr,def_ypp,def_ypr,in_off_ypp,in_off_ypr,in_def_ypp,in_def_ypr
index,,,,,,,,,,,,,,,,,,,,,
Bb,91.0,87.0,7.0,0.639722,0.511236,185.0,25.194595,22.135135,3.059459,11.891892,...,1.991228,2.377193,1.447024,-0.145820,-1.373909,-0.715125,1.282246,-0.185884,-1.145765,-0.519773
Z,93.0,87.0,6.0,-0.212700,0.516667,186.0,26.698925,24.591398,2.107527,12.306452,...,2.916667,3.425439,-0.022510,0.063420,-0.089176,0.122544,-0.050321,0.022426,-0.102516,0.085953
V,96.0,89.0,11.0,-0.150871,0.518919,196.0,26.081633,24.306122,1.775510,12.408163,...,2.964674,3.437500,-0.840767,-0.158621,0.077673,0.542051,-0.708416,-0.097712,0.068727,0.470657
U,91.0,91.0,17.0,-0.711747,0.500000,199.0,26.386935,24.849246,1.537688,11.733668,...,1.968232,2.465470,0.908054,0.365088,0.717107,0.740648,0.768461,0.314278,0.573093,0.701214
N,90.0,95.0,8.0,-0.716993,0.486486,193.0,24.481865,22.968912,1.512953,11.497409,...,2.517442,3.284884,-0.611201,1.430505,0.337744,-0.213360,-0.489121,1.220270,0.177813,-0.192148
G,81.0,92.0,5.0,-0.783297,0.468208,178.0,26.837079,25.522472,1.314607,11.955056,...,2.943787,3.440828,-0.502642,1.656460,0.733646,2.162420,-0.442720,1.359888,0.671271,1.690903
J,80.0,88.0,7.0,-0.098153,0.476190,175.0,23.257143,21.948571,1.308571,11.982857,...,2.664921,3.251309,-0.342486,-1.322569,-0.547983,-2.389866,-0.342614,-1.098307,-0.465153,-1.977476
S,93.0,86.0,12.0,-1.083833,0.519553,191.0,27.068063,25.921466,1.146597,11.832461,...,2.642045,3.255682,-0.013336,-0.892275,2.538546,-3.304371,-0.080363,-0.711263,2.064497,-2.720919
R,92.0,87.0,12.0,-0.234556,0.513966,191.0,24.015707,22.884817,1.130890,12.010471,...,2.838315,3.457880,-1.735332,0.489286,-0.061445,-0.317242,-1.445817,0.409849,-0.060232,-0.261653


In [1568]:
from sklearn.preprocessing import StandardScaler

scaled_array = StandardScaler().fit_transform(main_df)
scaled_df = pd.DataFrame(scaled_array, index=main_df.index, columns=main_df.columns)
corr = scaled_df.corr()
plt.figure(figsize=(12,8))
heat_map = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
plt.show()

ValueError: setting an array element with a sequence.

In [1300]:
from sklearn.linear_model import LinearRegression

X = scaled_df[['pass_pct', 'pass_1st', 'pass_2nd', 'pass_3rd', 'off_ypp', 'off_ypr']]
y = scaled_df[['pf_pg']]

reg = LinearRegression()
reg.fit(X, y)
print(reg.score(X, y))
print(reg.coef_)

0.319746445925865
[[-0.12263399  0.32567115 -0.60692633  0.45502792  0.47992917 -0.08823521]]
